In [1]:
import pandas as pd
import numpy as np

### Creating DF's

In [2]:
ruta_imports = 'C:/Users/juan-/Downloads/pharma/Bases/20220517_BD Imports Fracciones Biologicos.xlsx'
importsDB = pd.read_excel(ruta_imports)

In [3]:
ruta_medicamentos = 'C:/Users/juan-/Downloads/pharma/Bases/20220516_MEDICAMENTOS ONCOLOGICOS BIOLOGICOS_PW_AP_PW.xlsx'
MedicamentosDB = pd.read_excel(ruta_medicamentos, sheet_name='BIOLOGICOS')

In [4]:
imports = importsDB.copy()
medicines = MedicamentosDB.copy()

### Functions

In [5]:
def normalizar(text: str) -> str:
    """retorna la cadena del parametro sin acentos, en minuscula y quita espacios al principio o fin de la cadena"""
    a,b = 'áéíóúüÁÉÍÓÚÜ','aeiouuAEIOUU'
    trans = str.maketrans(a,b)
    try:
        text_normalized = text.translate(trans).lower().strip()
    except:
        print(f"error normalizando: {text}")
        text_normalized = "Error"
    return text_normalized

In [6]:
def normalizar_regimen(text: str) -> str:
    """retorna la cadena del parametro sin acentos, en minuscula y quita espacios al principio o fin de la cadena"""
    a,b = 'áéíóúüÁÉÍÓÚÜ','aeiouuAEIOUU'
    trans = str.maketrans(a,b)
    try:
        #quitamos acentos, espacios el inicio o final, y pasamos todo a minusculas
        text_normalized = text.translate(trans).lower().strip()
        #algunos regimenes diferian entre si por un punto al final del texto, por eso si un regimen termina con punto se le quita
        if text_normalized[-1] == '.':
            text_normalized = text_normalized[:-1]
        #algunas caracteres ascci se escapaban aún así, causado por los acentos por eso se quitan según sea necesario
        text_normalized =  text_normalized.replace('ã‰','e').replace('ã©','e').replace('ã','i').replace('ã“','o').replace('â€','').replace('â€œ','')
    except:
        #print(f"error normalizando: {text}")
        text_normalized = "Error"
    return text_normalized

In [14]:
def clasificar(row):
    """
    try to classify whether descripcion_comercial has name(N), brand(M), name and brand(NM),
    or if doesn't have name or brand(SN)"""
    try:
        for nombre in nombres_medicamentos:
            if nombre in row['descripcion_comercial']:
                for marca in marcas_medicamentos:
                    if marca in row['descripcion_comercial']:
                        return ['NM',nombre,marca]
                return ['N',nombre,'sin marca']

        for marca in marcas_medicamentos:
            if marca in row['descripcion_comercial']:
                return ['M','sin nombre',marca]

        return ['SN','sin nombre','sin marca']
        
    except Exception as e:
        print(e)
        return 'ERROR'

### Cleaning and renaming columnas(Normalización Data)

In [8]:
#Se borran columnas que no se ocupan en imports
imports.drop(columns=['Descripcion Partida Aduanera','NroSerie','Cod. Tributario', 'Ciudad Importador', 'Dirección Importador', 'Estado Importador','Ciudad Exportador', 
                        'Dirección Exportador', 'Qty 1','Unid 1','U$ FOB Unit 1'], inplace=True)

In [9]:
#renombro columnas para clarificar un poco más el nombre(aprovechando quito acentos y espacios)
imports.rename(columns={'Qty 2':'unidades',
                        'Unid 2':'tipo_unidad',
                        'U$ FOB Tot':'valor_total',
                        'U$ FOB Unit 2':'valor_unitario',
                        'País Origen':'pais_origen',
                        'Pais de Compra':'pais_compra',
                        'Vía':'via',
                        'Descripción Comercial':'descripcion_comercial'}, inplace=True)

In [ ]:
#se normalizan las columnas, minusculas, sin acentos y sin espacios al principio ni al final
imports['pais_compra_h'] = imports['pais_compra'].apply(normalizar)
imports['pais_origen_h'] = imports['pais_origen'].apply(normalizar)
imports['regimen_h'] = imports['Regimen'].apply(normalizar_regimen)
imports['descripcion_comercial'] = imports['descripcion_comercial'].apply(normalizar)

### Match y clasificacion nombre_y_marca

In [11]:
#creo dos sets de nombre y marca de medicamentps, set para evitar duplicados y acelerar la detección
nombres_medicamentos = set(medicines['MEDICAMENTO'].apply(normalizar))#73 nombres
marcas_medicamentos = set(medicines['NOMBRE COMERCIAL'].apply(normalizar))#111 marcas

In [15]:
#clasifico si tiene nombre, marca, ambos o ninguno, agrego nombre y marca en columnas separadas
imports[['nombre_y_marca','nombre','marca']] = imports.apply(clasificar,axis=1,result_type='expand')

In [19]:
matched_imports = imports.loc[(imports['nombre_y_marca'] == 'N') | (imports['nombre_y_marca'] == 'M') | (imports['nombre_y_marca'] == 'NM')].copy()

In [ ]:
matched_imports.head(2)

In [21]:
matched_imports.reset_index(drop=True, inplace=True)

In [22]:
matched_imports.drop(columns=['pais_origen', 'pais_compra','DUA','Aduana','Regimen'], inplace=True)

In [23]:
matched_imports.to_csv('matched_imports.csv',encoding='utf-8',index=False)